In [ ]:
import pprint
import scipy
from numpy.linalg import inv


def lu_decomposition(A):
    """Performs an LU Decomposition of A ."""
    n = len(A)

    # Create zero matrices for L and U
    L = scipy.zeros((n,n))
    U = scipy.zeros((n,n))


    # Perform the LU Decomposition
    for j in range(n):
        # All diagonal entries of L are set to unity
        L[j][j] = 1.0

        # LaTeX: u_{ij} = a_{ij} - \sum_{k=1}^{i-1} u_{kj} l_{ik}
        for i in range(j+1):
          s1=0;
          for k in range(i):
            s1 = s1+(U[k][j] * L[i][k])
          U[i][j] = A[i][j] - s1

        # LaTeX: l_{ij} = \frac{1}{u_{jj}} (a_{ij} - \sum_{k=1}^{j-1} u_{kj} l_{ik} )
        for i in range(j, n):
          s2 = 0;
          for k in range(j):
            s2 = s2+ (U[k][j] * L[i][k])
            L[i][j] = (A[i][j] - s2) / U[j][j]

    return (L, U)


#A = [[1,3,6], [2,5,3], [5,2,1]]
#L = scipy.array([[1,0,0],[2,1,0],[2,2,1]])
#U = scipy.array([[1,3,6],[0,1,3],[0,0,2]])
L = scipy.array([[1,0,0],[2,1,0],[2,2,1]])
U = scipy.array([[1,3,6],[0,1,3],[0,0,1]])

A = scipy.dot(L,U);
#A = scipy.array([[1,3,6],[2,5,3],[5,2,1]])
L, U = lu_decomposition(A)
inva = inv(A)

print ("A:")
pprint.pprint(A)

print ("L:")
pprint.pprint(L)

print ("U:")
pprint.pprint(U)

print ("A^(-1):")
pprint.pprint(inva)

A:
array([[ 1,  3,  6],
       [ 2,  7, 15],
       [ 2,  8, 19]])
L:
array([[1., 0., 0.],
       [2., 1., 0.],
       [2., 2., 1.]])
U:
array([[1., 3., 6.],
       [0., 1., 3.],
       [0., 0., 1.]])
A^(-1):
array([[13., -9.,  3.],
       [-8.,  7., -3.],
       [ 2., -2.,  1.]])


<ipython-input-18-b09e8c73c17e>:36: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  L = scipy.array([[1,0,0],[2,1,0],[2,2,1]])
<ipython-input-18-b09e8c73c17e>:37: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  U = scipy.array([[1,3,6],[0,1,3],[0,0,1]])
<ipython-input-18-b09e8c73c17e>:39: DeprecationWarning: scipy.dot is deprecated and will be removed in SciPy 2.0.0, use numpy.dot instead
  A = scipy.dot(L,U);
<ipython-input-18-b09e8c73c17e>:11: DeprecationWarning: scipy.zeros is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros instead
  L = scipy.zeros((n,n))
<ipython-input-18-b09e8c73c17e>:12: DeprecationWarning: scipy.zeros is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros instead
  U = scipy.zeros((n,n))


Assim modelamos agora as regiões do nosso espaço em questão que é divida em 4:


---


0.   Músculo
1.   Tumor



---





In [ ]:
%%writefile tecidos.h
#ifndef TECIDOS_H
#define TECIDOS_H

void determinaTecidos(int *tecidos, int n);
void salvaMatriz(int *tecidos, int n);

#endif //TECIDOS_H

Writing tecidos.h


In [ ]:
%%writefile tecidos.cpp
#include <iostream>
using namespace std;
#include <fstream>
#include <vector>

#include "tecidos.h"

void determinaTecidos(int *tecidos, int n) {
  //0 -> musculo
  //1 -> tumor
  //tumor esfera de raio 13mm -> consideramos um quadrado nessa abordagem

  for(int i=0; i<n; i++) {
    tecidos[i] = (i > 37 && i < 63) ?  1 : 0;
  }
}

void salvaMatriz(int *tecidos, int n) {
  ofstream arquivo("tecidos.txt");

  // Verificando se o arquivo foi aberto corretamente
  if (!arquivo.is_open()) {
      cerr << "Erro ao abrir o arquivo." << endl;
  }

  // Escrevendo os elementos da matriz no arquivo
  for (int i = 0; i < n; ++i) {
      arquivo << tecidos[i] << " ";
  }

  // Feche o arquivo
  arquivo.close();

  cout << "Matriz salva com sucesso em 'tecidos.txt'." << endl;
}

Writing tecidos.cpp


Temos que a condutividade térmica varia de acordo com o tecido, porém vamos considerá-la constante inicialmente:

\begin{equation*}
  k_{muscle} = k_{tumor} = 0.5\\
\end{equation*}

A geração de calor metabólico varia de acordo com a região tratada, temos assim então:

\begin{equation*}
  Q_{m_{muscle}} = 420.0 \\
  Q_{m_{tumor}} = 4200.0
\end{equation*}

A taxa de perfusão sanguínea varia de acordo com o tecido e com a temperatura que esse se encontra, dessa forma temos que:

\begin{equation*}
  W_{muscle} = 0.45\\
  W_{tumor} = 0.833\\
\end{equation*}

A densidade dos tecidos são diferentes uma das outras, dessa forma definimos para a mesma a seguinte variação, nesse caso os tecidos tem o mesmo valor coincidentemente:

\begin{equation*}
  \rho_{muscle} = \rho_{tumor} = 1000\\
\end{equation*}

O calor específico dos tecidos são diferentes uma das outras, dessa forma definimos para a mesma a seguinte variação:

\begin{equation*}
  c_{muscle} = 3800 \\
  c_{tumor} = 4200\\
\end{equation*}

In [ ]:
%%writefile variaveis.h
#ifndef VARIAVEIS_H
#define VARIAVEIS_H

void setQm(int *qm);
void setW(float *w);
void setP(int *p);
void setC(int *c);

#endif //VARIAVEIS_H

Writing variaveis.h


In [ ]:
%%writefile variaveis.cpp
#include <iostream>
using namespace std;

#include "variaveis.h"

void setQm(int *qm) {
  qm[0]=420;
  qm[1]=4200;
}

void setW(float *w) {
  w[0] = 0.45;
  w[1] = 0.833;
}

void setP(int *p) {
  p[0]=1200;
  p[1]=1200;
}

void setC(int *c) {
  c[0] = 3800;
  c[1] = 4200;
}


Writing variaveis.cpp


Equação a ser modelada:

\begin{equation*}
  \begin{cases}
    \displaystyle \nabla \cdot k \nabla T + w_b c_b(T_a - T) + Q_m = 0 \text{ para } \Omega\\\\
    \displaystyle k \nabla T \cdot\vec{n} = 0 \text{ para } \partial \Omega_N\\\\
    \displaystyle T  = 37 \text{ para } \partial \Omega_D\\\\
    \displaystyle T(.,0) = 37
  \end{cases}
\end{equation*}

\begin{equation*}
  \text { onde } T : \Omega \times  I \rightarrow \mathbb{R^+} \text { representa o campo de temperatura do tecido } \\
  c_b : \Omega \times  I \rightarrow \mathbb{R^+} \text { representa o calor específico do sangue } \\
  k : \Omega \times  I \rightarrow \mathbb{R^+} \text { na qual a condutividade térmica é considerada isotrópica, mas não homogênea } \\
  w_b:  \Omega \times {R^+} \times  I \rightarrow \mathbb{R^+} \text { taxa de perfusão sanguínea } \\
  T_a \text { temperatura arterial } \\
  Q_m : \Omega \times  I \rightarrow \mathbb{R^+} \text { representa a geração de calor metabólico } \\
  f : \Omega \times  I \rightarrow \mathbb{R^+} \text { temperaturaprescrita(Dirichelet) ou fluxo (Neumann ou Robin} \\
  \alpha , \beta : \Omega \times  I \rightarrow \mathbb{R^+} \text { sendo o vetor normal unitário } \\
\end{equation*}

Considerando o caso mais geral de equação linear de segunda ordem, temos:

\begin{equation}
  a(x)u''(x) + b(x)u'(x) + c(x)u(x) = f(x)
\end{equation}

Assim transferindo e adapatando para a equação de Pennes simplificada divimos a equação da seguinte forma:

\begin{equation}
   \nabla \cdot k \nabla T - Tw_bc_b = -Q_m - w_bc_bT_a
\end{equation}

Na qual o primeiro termo, equivale ao ${a(x)}$ da equação geral:

\begin{equation}
  k \nabla^2 T
\end{equation}

O segundo termo equivale a ${c(x)}$:

\begin{equation}
  - w_bc_bT
\end{equation}

Já o lado direito representa f(x):

\begin{equation}
  -Q_m - w_bc_bT_a
\end{equation}

Já ${b(x)}$ vai ser nulo, já que não temos nenhuma derivada da segunda em questão

A matriz A, vai seguir o seguinte formato dadas as condições acima:

\begin{equation*}
  A =
  \begin{bmatrix}
    h^2w_bc_b -2k & k &  &  &  &\\
    k & h^2w_bc_b -2k & k &  &  & \\
    &  k &  h^2w_bc_b -2k & k &  & \\
    &  &  ... & ... & ... &\\
    &  &  &  k & h^2w_bc_b -2k & k\\
    &  &  &  &  k & h^2w_bc_b -2k\\
  \end{bmatrix}
\end{equation*}

Já a matriz f, seguirá o seguite formato:

\begin{equation*}
  f =
  \begin{bmatrix}
   h^2(-Q_m -w_bc_bT_a) - u_a\\
   h^2(-Q_m -w_bc_bT_a) \\
   h^2(-Q_m -w_bc_bT_a) \\
   ...  \\
   h^2(-Q_m -w_bc_bT_a)  \\
   h^2(-Q_m -w_bc_bT_a) - u_b \\
  \end{bmatrix}
\end{equation*}

Nesse sentido vamos seguir o seguinte algoritmo para resolução do problema:



1.   Montar o sistema $Au = F$
2.   Resolver esse sistema
3.   Aplicar condição de contorno em u

Para isso aplicamos o método de decomposição LU sem pivoteamento

In [ ]:
%%writefile lu.h
#ifndef LU_H
#define LU_H

int detInd(int i, int j, int n);
void montaA(float *A,  int n, float h, float k, float *w, float cb, int *qm, int Ta, int *tecidos);
void montaF(float *F, int n, float h, int *qm, float *w, float cb, int Ta, int *tecidos);
void resolucao(float A[], float F[], float *u, int n);

#endif //LU_H

Overwriting lu.h


In [ ]:
%%writefile lu.cpp
#include <iostream>
using namespace std;
#include <fstream>
#include <vector>

#include "variaveis.h"

int detInd(int i, int j, int n) {
  return (i < 100 && j < 100) ? i*n + j : -1;
}

void montaA(float *A,  int n, float h, float k, float *w, float cb, int *qm, int Ta, int *tecidos) {
    for(int i=0; i<n; i++) {
      for(int j=0; j<n; j++) {
        if(j>=i-1 && j<i+2) {
           A[detInd(i,j,n)] = i==j ?  ((h*h*w[tecidos[i]]*cb) - 2*k) : k;
        }
        else A[detInd(i,j,n)] = 0;
      }
    }
}

void montaF(float *F, int n, float h, int *qm, float *w, float cb, int Ta, int *tecidos) {
  int ua = 37; //->condicao de contorno esquerda
  int ub = 37; //->condicao de contorno direita

  for(int i=1; i<n; i++) {
    F[i] = h*h*(-qm[tecidos[i]] -w[tecidos[i]]*cb*Ta);
  }

  F[0] = h*h*(-qm[tecidos[0]] -w[tecidos[0]]*cb*Ta) - ua;
  F[n-1] = h*h*(-qm[tecidos[n-1]] -w[tecidos[n-1]]*cb*Ta) - ub;

  cout << endl;
  cout << endl;
}

void resolucao(float *A, float *F, float *u, int n) {
  float L[n][n];
  float U[n][n];

  //MONTAR AS MATRIZES L E U
  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      U[i][j] = 0;
      L[i][j] = 0;
    }
  }

  for(int i=0; i<n; i++) {
    L[i][i] = 1;

    for(int j=i; j<n; j++) {
      U[i][j] = A[detInd(0,j-i,n)];
    }

    for(int j=i+1; j<n; j++) {
      L[j][i] = A[detInd(j,i,n)] / A[detInd(i,i,n)];
    }
  }


  //DUVIDA NO PROCESSO DE RESOLUCAO CONCEITO
  //RESOLVER O SISTEMA Ly = F
  float y[n];

  for(int i=0; i < n; i++) {
        y[i] = F[i];
        for(int j=0; j<i; j++) {
            y[i] -= L[i][j] * y[j];
        }
        y[i] = y[i] / L[i][i];
    }

    //RESOLVER O SISTEMA Uu = y
    for(int i= n-1; i >= 0; i--) {
        u[i] = y[i];
        for (int j= i+1; j < n; j++) {
            u[i] -= U[i][j] * u[j];
        }
        u[i] = u[i] / U[i][i];
    }

    ofstream arquivo("resolucao.txt");

    // Verificando se o arquivo foi aberto corretamente
    if (!arquivo.is_open()) {
        cerr << "Erro ao abrir o arquivo." << endl;
    }

    // Escrevendo os elementos da matriz no arquivo
    for (int i = 0; i < n; ++i) {
        arquivo << u[i] << " ";
    }

    // Feche o arquivo
    arquivo.close();

    cout << "Matriz salva com sucesso em 'resolucao.txt'." << endl;
}

Overwriting lu.cpp


In [ ]:
%%writefile main.cpp
#include <iostream>
using namespace std;
#include <string>
#include <cmath>

#include "tecidos.h"
#include "variaveis.h"
#include "lu.h"

int main()
{

    int n = 3; //tamanho do dominio
    float h = 0.001; // (0.1 - 0)/100

    //SE O DOMINIO FOSSE 2D
    //int *tecidos = new int[tamX*tamY];
    //int tamX = 51;
    //int tamY = 101; int n
    //determinaTecidos(tecidos, tamX, tamY);
    //salvaMatriz(tecidos,tamX,tamY);

    //DOMINIO 1D
    int *tecidos = new int[n];
    determinaTecidos(tecidos, n);
    salvaMatriz(tecidos, n);

    float k = 0.5;
    int *qm = new int[2];
    float *w = new float[2];
    int *p = new int[2];
    int *c = new int[2];

    setQm(qm);
    setW(w);
    setP(p);
    setC(c);

    float cb = 4200; //calor especifico do sangue (constante)
    int Ta = 37; //temperatura inicial (constante)

    float A[3][3] = {
      {1,0,1}, {1,1,0}, {2,3,1}
    };

    float F[3] = {0,1,1};
    //float *F = new float[n];
    float *u = new float[n];

    //montaA(A, n, h, k, w, cb, qm, Ta, tecidos);
    //montaF(F, n, h, qm, w, cb, Ta, tecidos);
    resolucao(A, F, u, n);

    delete [] tecidos;
    delete [] qm;
    delete [] w;
    delete [] p;
    delete [] c;
    //delete [] A;

    return 0;
}

Overwriting main.cpp


In [ ]:
!g++ tecidos.cpp \
  variaveis.cpp \
  lu.cpp \
  main.cpp \
  -o decompLu

main.cpp: In function ‘int main()’:
main.cpp:52:15: error: cannot convert ‘float (*)[3]’ to ‘float*’
   52 |     resolucao(A, F, u, n);
      |               ^
      |               |
      |               float (*)[3]
In file included from main.cpp:8:
lu.h:7:23: note:   initializing argument 1 of ‘void resolucao(float*, float*, float*, int)’
    7 | void resolucao(float *A, float *F, float *u, int n);
      |                ~~~~~~~^


In [ ]:
!./decompLu

Matriz salva com sucesso em 'tecidos.txt'.


Matriz salva com sucesso em 'tecidos.txt'.


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d, Axes3D
import numpy as np
import math

from google.colab import files
from matplotlib.figure import projections

plt.rcParams['figure.figsize'] = [8, 6]

# Carregar a matriz do arquivo
# matriz = np.loadtxt('matriz.txt')
matriz = np.loadtxt('tecidos.txt')

# Exibir a matriz
print("Matriz de tecidos:")
print(matriz)

resolucao = np.loadtxt('resolucao.txt')

# Exibir a matriz
print("Matriz de resolucao:")
print(resolucao)

Matriz de tecidos:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
Matriz de resolucao:
[49.6778   25.0272   12.6785    6.4925    3.39363   1.84126   1.06361
  0.674045  0.478895  0.381135  0.332162  0.307629  0.29534   0.289183
  0.286099  0.284554  0.283781  0.283393  0.283199  0.283102  0.283053
  0.283029  0.283019  0.283015  0.283017  0.283026  0.283047  0.283089
  0.283173  0.283341  0.283677  0.284349  0.285689  0.288363  0.293703
  0.304361  0.325638  0.368112  0.452899  0.495555  0.516958  0.527697
  0.533086  0.535789  0.537145  0.537826  0.538167  0.538337  0.53842
  0.538458  0.538469  0.538459  0.538422  0.53834   0.538173  0.537837
  0.537166  0.535826  0.533151  0.52781   0.517148  0.495866  0.45338
  0.368571  0.3258

In [ ]:
%%writefile tecidos.h
#ifndef TECIDOS_H
#define TECIDOS_H

int detInd(int i, int j, int tamY);
void determinaTecidos(int *tecidos, int tamX, int tamY);
void salvaMatriz(int *tecidos, int tamX, int tamY);

#endif //TECIDOS_H

Writing tecidos.h


In [ ]:
%%writefile tecidos.cpp
#include <iostream>
using namespace std;
#include <fstream>
#include <vector>

#include "tecidos.h"

int detInd(int i, int j, int tamY) {
  return (i < 51 && j<101) ? i*tamY + j : -1;
}

void determinaTecidos(int *tecidos, int tamX, int tamY) {
  //0 -> musculo
  //1 -> tumor
  //tumor esfera de raio 13mm -> consideramos um quadrado nessa abordagem
  //centro (20,50)mm

  for(int i=0; i<tamX; i++) {
    for(int j=0; j<tamY; j++) {
      (i > 19 && i<45 && j>37 && j<63) ? tecidos[detInd(i,j,tamY)] = 1 : tecidos[detInd(i,j,tamY)] = 0;
    }
  }
}

void salvaMatriz(int *tecidos, int tamX, int tamY) {
    // Matriz que vai receber os valores dos tecidos
    vector<vector<int>> matriz;

    // Inicializar a matriz que vai ser salva com elementos
    matriz.resize(tamX);
    for (int i = 0; i < tamX; i++) {
      matriz[i].resize(tamY);
      for (int j = 0; j < tamY; j++) {
        matriz[i][j] =  tecidos[detInd(i,j,tamY)];
      }
    }

    // Salvar a matriz de tecidos em um arquivo para ser importada posteriormente
    ofstream arquivo("matriz.txt");
    if (arquivo.is_open()) {
      for (const auto& linha : matriz) {
        for (const auto& elemento : linha) {
          arquivo << elemento << " ";
        }
        arquivo << "\n";
      }
      arquivo.close();
      cout << "C++: Matriz de tecidos salva em matriz.txt" << endl;
    } else {
      cerr << "C++: Erro ao abrir o arquivo" << endl;
    }
}

Writing tecidos.cpp
